In [ ]:
%tensorflow_version 1.x
# change the tensorflow version

TensorFlow 1.x selected.


In [ ]:
import tensorflow
print(tensorflow.__version__)
# check the version of tensorflow

1.15.2


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

In [ ]:
from google.colab import files
upload=files.upload()
# to load the 'train.csv' file

Saving train.csv to train.csv


In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
x=df.iloc[:42000,1:].values

In [ ]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
x.shape

(42000, 784)

In [ ]:
x1=tf.placeholder(tf.float32,shape=[None,28*28])
# here we must use none in placeholder because we use latter mini-batch gradient so we don't know how many example of pixel image we feed into cost function together
x_img=tf.reshape(x1,shape=[-1,28,28,1])
# here we use -1 in batch to take batch pixel example dynamically
x=np.array(x/255)
# here we reshape the pixel image and normalize the image to get best prediction

In [ ]:
def ultimate_img(img):
  k1=tf.Variable(tf.truncated_normal(shape=[3,3,1,10],stddev=0.1))
  c1=tf.Variable(tf.zeros([10]))
  a1=tf.nn.conv2d(img,filter=k1,strides=[1,1,1,1],padding='SAME')
  a1=a1+c1
  b1=tf.nn.max_pool(a1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
  a2=tf.nn.conv2d(b1,filter=k1,strides=[1,1,1,1],padding='SAME')
  a2=a2+c1
  b2=tf.nn.max_pool(a2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
  return b2
# this is the step where the picture pass through convolution and pooling step

In [ ]:
y=df.iloc[:42000,:1].values
Y=tf.placeholder(tf.float32,shape=[None,10])
from keras.utils import to_categorical
y1 = to_categorical(y)
# we use this code to make the labels one-hot-encoded
y_actual=tf.argmax(Y,1)

In [ ]:
y1.shape

(42000, 10)

In [ ]:
x=df.iloc[:42000,1:].values
x1=tf.placeholder(tf.float32,shape=[None,28*28])
x_img=tf.reshape(x1,shape=[-1,28,28,1])
x=np.array(x/255)
def ultimate_img(img):
  k1=tf.Variable(tf.truncated_normal(shape=[3,3,1,10],stddev=0.1))
  c1=tf.Variable(tf.zeros([10]))
  a1=tf.nn.conv2d(img,filter=k1,strides=[1,1,1,1],padding='SAME')
  a1=a1+c1
  b1=tf.nn.max_pool(a1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
  a2=tf.nn.conv2d(b1,filter=k1,strides=[1,1,1,1],padding='SAME')
  a2=a2+c1
  b2=tf.nn.max_pool(a2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
  return b2
y=df.iloc[:42000,:1].values
Y=tf.placeholder(tf.float32,shape=[None,10])
from keras.utils import to_categorical
y1 = to_categorical(y)
y_actual=tf.argmax(Y,1)
a=ultimate_img(x_img)
l=tf.reshape(a,shape=[-1,490])
w1=tf.Variable(tf.truncated_normal(shape=[490,500],stddev=0.1))
b1=tf.Variable(tf.zeros([1,500]))
w2=tf.Variable(tf.truncated_normal(shape=[500,500],stddev=0.1))
b2=tf.Variable(tf.zeros([1,500]))
w3=tf.Variable(tf.truncated_normal(shape=[500,500],stddev=0.1))
b3=tf.Variable(tf.zeros([1,500]))
w4=tf.Variable(tf.truncated_normal(shape=[500,10],stddev=0.1))
b4=tf.Variable(tf.zeros([1,10]))
z1=tf.nn.relu(tf.matmul(l,w1)+b1)
dropped=tf.nn.dropout(z1,keep_prob=0.6)
z2=tf.nn.relu(tf.matmul(dropped,w2)+b2)
dropped1=tf.nn.dropout(z2,keep_prob=0.6)
z3=tf.nn.relu(tf.matmul(dropped1,w3)+b3)
dropped2=tf.nn.dropout(z2,keep_prob=0.6)
z4=tf.nn.relu(tf.matmul(dropped2,w4)+b4)
y_=tf.nn.softmax(z4,1)
y_pred0=tf.argmax(y_,1)
# we use argmax for take the result of hiest probability
cross_entropy=tf.nn.softmax_cross_entropy_with_logits_v2(logits=z4,labels=Y)
model_cost=tf.reduce_mean(cross_entropy)
learning_rate=0.0001
optimizer=tf.train.AdamOptimizer(learning_rate).minimize(model_cost)
model_pred=tf.equal(y_pred0,y_actual)
model_accuracy=tf.reduce_mean(tf.cast(model_pred,tf.float32))
init=tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)
cost_history=[]
s=[]
for epoch in range(50):
  k=0
  for i in range(0,42000,80):
    p=y1[i:i+80,:]
    m=x[i:i+80,:]
    sess.run(model_pred,feed_dict={x1:m,Y:p})
    sess.run(optimizer,feed_dict={x1:m,Y:p})
    k+=sess.run(model_accuracy,feed_dict={x1:m,Y:p})
    cost_=sess.run(model_cost,feed_dict={x1:m,Y:p})
    cost_history=np.append(cost_history,cost_)
  su=sess.run(y_pred0,feed_dict={x1:x,Y:y1})
  s=np.append(s,su)
  # s is a list were we can store result every cost ,with better and better accuracy we can store better and better result of prediction every time
  k/=int(len(y1)/80)
  print("reached epoch",epoch,"cost J=",cost_,"accuracy=",k)
  # at every bath we get a some level of accuracy and by avaraging the hole accuracy we find a best accuracy for a hole training input in every epoch

/tensorflow-1.15.2/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


reached epoch 0 cost J= 0.6488366 accuracy= 0.6292857148108029
reached epoch 1 cost J= 0.27387416 accuracy= 0.8425476188886734
reached epoch 2 cost J= 0.17449121 accuracy= 0.9124047624497187
reached epoch 3 cost J= 0.12302922 accuracy= 0.9306666663714818
reached epoch 4 cost J= 0.08728783 accuracy= 0.9416190461885361
reached epoch 5 cost J= 0.11382244 accuracy= 0.9485952371642703
reached epoch 6 cost J= 0.07731379 accuracy= 0.9549523792948041
reached epoch 7 cost J= 0.093839645 accuracy= 0.9591190476644607
reached epoch 8 cost J= 0.0741338 accuracy= 0.9624761919748216
reached epoch 9 cost J= 0.082443625 accuracy= 0.9658095253081549
reached epoch 10 cost J= 0.07896295 accuracy= 0.9687619071915036
reached epoch 11 cost J= 0.06731879 accuracy= 0.9699285741079421
reached epoch 12 cost J= 0.072366685 accuracy= 0.9723095272836231
reached epoch 13 cost J= 0.034965128 accuracy= 0.9730238127708435
reached epoch 14 cost J= 0.044646826 accuracy= 0.9743333376021612
reached epoch 15 cost J= 0.07380

In [ ]:
from google.colab import files
upload=files.upload()
# Let's load the 'test.csv' value

Saving test.csv to test.csv


In [ ]:
df1=pd.read_csv('test.csv')

In [ ]:
df1

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,183,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,252,20,0,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
BATCH_SIZE=86
x_test=df1.iloc[:28000].values
predicted_lables = np.zeros(28000)
for i in range(0,28000//BATCH_SIZE):
    predicted_lables[i*BATCH_SIZE : (i+1)*BATCH_SIZE] = sess.run(y_pred0,feed_dict={x1: x_test[i*BATCH_SIZE : (i+1)*BATCH_SIZE]})
# we can predict result by this process when we only feed pixel input and want to predict labels

In [ ]:
predicted_lables=predicted_lables.reshape(28000,1)
#Reshape the predicted result to confirm that the shape is (28000,1)

In [ ]:
y_actual1=df.iloc[29400:42000,:1].values

In [ ]:
df2=pd.DataFrame(predicted_lables)
# convert the predicted array in dataframe

In [ ]:
df2

,0
0,2.0
1,0.0
2,9.0
3,9.0
4,3.0
...,...
27995,0.0
27996,0.0
27997,0.0
27998,0.0


In [ ]:
l=[]
for i in range(1,28001):
  l.append(i)

In [ ]:
dfi=pd.DataFrame(l)
# make an index dataframe to add it with predicted_labels

In [ ]:
dfi

,0
0,1
1,2
2,3
3,4
4,5
...,...
27995,27996
27996,27997
27997,27998
27998,27999


In [ ]:
krame=[dfi,df2]

In [ ]:
ki=pd.concat(krame,axis=1)

In [ ]:
ki=pd.DataFrame(ki)

In [ ]:
ki

,0,0
0,1,2.0
1,2,0.0
2,3,9.0
3,4,9.0
4,5,3.0
...,...,...
27995,27996,0.0
27996,27997,0.0
27997,27998,0.0
27998,27999,0.0


In [ ]:
from google.colab import drive
drive.mount('drive')
# to mounting the drive

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
ki.to_csv('digi_test(1).csv')
!cp data.csv "drive/My Drive/"
# to save the result in my drive of google colab

In [ ]:
acc=np.equal(predicted_lables,y_actual1)
ac=np.mean(acc)
# if we separate the training set into training and test set then after training, by only using the testing input we predict the labels of image pixel and compare with
#actual predicted labels to see what accuracy it gain in testing.